In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import string
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Flatten, GlobalMaxPooling1D
from tensorflow.keras.models import Model
import random

# Loading Content

Content is a JSON file containing tag, input and response. What we going to do is predict tag based on input and give random answer from responses.

In [2]:
intents = json.loads(open('./intents.json').read())

In [3]:
intents


{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hey :-) My name is Jarvis!',
    'Hello, thanks for visiting. My name is Jarvis!',
    'Hi there, My name is Jarvis!. What can I do for you?',
    'Hi there, My name is Jarvis! How can I help?']},
  {'tag': 'personal',
   'patterns': ['How are you?', 'How are you doing?', 'How is your day?'],
   'responses': ["I'm good, all's good, thanks. How about you?"]},
  {'tag': 'user response',
   'patterns': ["I'm good",
    'Im good',
    'Im doing good',
    'I am good',
    'I am okay'],
   'responses': ['Great to hear you are doing good.']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting. Hope I was able to help!',
    'Have a nice day. Hope I was able to help!']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    "Thank's a lot!",
    'thnx',
    'ok thn

In [4]:
tags = []
inputs = []
responses = {}
for intent in intents['intents']:
    responses[intent['tag']] = intent["responses"]
    for lines in intent['patterns']:
        inputs.append(lines)
        tags.append(intent['tag'])


print("this is tags",tags)
print("this is input",inputs)
print("this is responses",responses)

this is tags ['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'personal', 'personal', 'personal', 'user response', 'user response', 'user response', 'user response', 'user response', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'items', 'items', 'items', 'items', 'items', 'payments', 'payments', 'payments', 'payments', 'payments', 'payments', 'track', 'track', 'track', 'track', 'track', 'delivery', 'delivery', 'delivery', 'delivery', 'delivery', 'delivery', 'delivery', 'confirm', 'confirm', 'confirm', 'confirm', 'delay', 'delay', 'delay', 'delay', 'reasons', 'reasons', 'reasons', 'reasons', 'reasons', 'more', 'more', 'more', 'more', 'more', 'more', 'more', 'more', 'more', 'more', 'more', 'cancel', 'cancel', 'cancel', 'cancel', 'cancel', 'refund', 'refund', 'refund', 'refund', 'refund status', 'refund status', 'refund status', 'refund status', 'refund status', 'refund', 'refund', 'refund', 'refund', 'return', 'return', 'return'

# Data preprocessing

In [5]:
data = pd.DataFrame({"inputs":inputs,
                    "tags":tags})

In [6]:
data

,inputs,tags
0,Hi,greeting
1,Hey,greeting
2,Is anyone there?,greeting
3,Hello,greeting
4,Good day,greeting
...,...,...
101,delete my account,profile info
102,Can I download a mobile app for this?,mobile app
103,Mobile app,mobile app
104,Download mobile app,mobile app


Removing punctuations and converting text to lower case

In [7]:
data["inputs"] = data["inputs"].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])

In [8]:
data["inputs"] = data["inputs"].apply(lambda wrd: ''.join(wrd))

In [9]:
data

,inputs,tags
0,hi,greeting
1,hey,greeting
2,is anyone there,greeting
3,hello,greeting
4,good day,greeting
...,...,...
101,delete my account,profile info
102,can i download a mobile app for this,mobile app
103,mobile app,mobile app
104,download mobile app,mobile app


# Tokenising and Padding

In [10]:
tokenizer = Tokenizer(num_words = 2000)

In [11]:
tokenizer.fit_on_texts(data["inputs"])

In [12]:
train = tokenizer.texts_to_sequences(data["inputs"])

In [13]:
train

[[81],
 [82],
 [6, 83, 26],
 [84],
 [21, 52],
 [7, 11, 10],
 [7, 11, 10, 53],
 [7, 6, 54, 52],
 [37, 21],
 [37, 21],
 [37, 53, 21],
 [1, 55, 21],
 [1, 55, 85],
 [86],
 [87, 10, 88],
 [89],
 [56],
 [90, 10],
 [91, 92],
 [56, 13, 93],
 [57],
 [94, 57],
 [95, 22, 8, 10, 96],
 [14, 97, 98, 22, 11, 26],
 [14, 8, 10, 99],
 [14, 8, 10, 100],
 [14, 4, 1, 101],
 [8, 10, 27, 102, 103],
 [8, 10, 104, 105],
 [4, 1, 58, 106, 59],
 [11, 10, 59, 107],
 [14, 11, 54, 38, 108],
 [7, 8, 1, 58],
 [7, 4, 1, 28, 2, 3],
 [1, 12, 5, 28, 2, 3],
 [4, 1, 28, 2, 3],
 [28, 3],
 [60, 6, 2, 3],
 [7, 39, 61, 15, 27],
 [7, 39, 61, 62, 27],
 [109, 110, 63, 111, 2, 15],
 [18, 8, 1, 23, 2, 15],
 [19, 6, 2, 3, 16, 40, 64],
 [19, 6, 2, 3, 112, 113, 114, 115, 65, 5, 40],
 [60, 6, 2, 3],
 [2, 38, 41, 116, 117, 42, 1, 118, 23, 66, 3, 67, 14, 119, 1, 8],
 [38, 41, 120, 42, 43, 67],
 [3, 41, 16, 68],
 [44, 121, 42, 3, 6, 16, 68],
 [19, 6, 2, 3, 69],
 [70, 2, 3, 23, 69],
 [19, 6, 2, 3, 16, 40, 64],
 [18, 8, 1, 23, 2, 15],
 [71, 

In [14]:
x_train = pad_sequences(train)

# Encoding the output

In [15]:
le = LabelEncoder()

In [16]:
y_train = le.fit_transform(data["tags"])

In [17]:
input_shape = x_train.shape[1]
unique_words = len(tokenizer.word_index)
output_length = le.classes_.shape[0]

# Building the neural network 

In [18]:
model = tf.keras.Sequential()
model.add(Input(shape=(input_shape,)))
model.add(Embedding(unique_words+1, 10, input_length= (input_shape,)))
model.add(LSTM(10, return_sequences=True))
model.add(Flatten())
model.add(Dense(units= 10, activation='relu'))
model.add(Dense(units= 5, activation='relu'))
model.add(Dense(units= output_length, activation='softmax'))

In [19]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer ='adam', metrics=['accuracy'])

In [20]:
jarvis = model.fit(x_train, y_train, epochs=300)

Epoch 1/300
4/4 [==============================] - 5s 18ms/step - loss: 2.9957 - accuracy: 0.0377
Epoch 2/300
4/4 [==============================] - 0s 26ms/step - loss: 2.9942 - accuracy: 0.0566
Epoch 3/300
4/4 [==============================] - 0s 25ms/step - loss: 2.9926 - accuracy: 0.0472
Epoch 4/300
4/4 [==============================] - 0s 23ms/step - loss: 2.9915 - accuracy: 0.0472
Epoch 5/300
4/4 [==============================] - 0s 22ms/step - loss: 2.9901 - accuracy: 0.0472
Epoch 6/300
4/4 [==============================] - 0s 23ms/step - loss: 2.9886 - accuracy: 0.0472
Epoch 7/300
4/4 [==============================] - 0s 22ms/step - loss: 2.9875 - accuracy: 0.0472
Epoch 8/300
4/4 [==============================] - 0s 22ms/step - loss: 2.9855 - accuracy: 0.0472
Epoch 9/300
4/4 [==============================] - 0s 21ms/step - loss: 2.9839 - accuracy: 0.0472
Epoch 10/300
4/4 [==============================] - 0s 17ms/step - loss: 2.9821 - accuracy: 0.0472
Epoch 11/300
4/4 [=

4/4 [==============================] - 0s 12ms/step - loss: 1.8384 - accuracy: 0.3962
Epoch 84/300
4/4 [==============================] - 0s 14ms/step - loss: 1.8201 - accuracy: 0.3962
Epoch 85/300
4/4 [==============================] - 0s 15ms/step - loss: 1.8023 - accuracy: 0.3774
Epoch 86/300
4/4 [==============================] - 0s 16ms/step - loss: 1.7963 - accuracy: 0.3679
Epoch 87/300
4/4 [==============================] - 0s 18ms/step - loss: 1.8745 - accuracy: 0.3491
Epoch 88/300
4/4 [==============================] - 0s 16ms/step - loss: 1.7924 - accuracy: 0.3868
Epoch 89/300
4/4 [==============================] - 0s 13ms/step - loss: 1.7711 - accuracy: 0.3868
Epoch 90/300
4/4 [==============================] - 0s 11ms/step - loss: 1.7812 - accuracy: 0.3868
Epoch 91/300
4/4 [==============================] - 0s 17ms/step - loss: 1.7659 - accuracy: 0.3868
Epoch 92/300
4/4 [==============================] - 0s 15ms/step - loss: 1.7247 - accuracy: 0.3868
Epoch 93/300
4/4 [=====

4/4 [==============================] - 0s 25ms/step - loss: 0.9546 - accuracy: 0.6981
Epoch 166/300
4/4 [==============================] - 0s 14ms/step - loss: 0.9417 - accuracy: 0.6887
Epoch 167/300
4/4 [==============================] - 0s 17ms/step - loss: 0.9349 - accuracy: 0.6792
Epoch 168/300
4/4 [==============================] - 0s 14ms/step - loss: 0.9311 - accuracy: 0.7170
Epoch 169/300
4/4 [==============================] - 0s 17ms/step - loss: 0.9177 - accuracy: 0.7264
Epoch 170/300
4/4 [==============================] - 0s 15ms/step - loss: 0.9179 - accuracy: 0.7547
Epoch 171/300
4/4 [==============================] - 0s 22ms/step - loss: 0.9421 - accuracy: 0.6887
Epoch 172/300
4/4 [==============================] - 0s 12ms/step - loss: 0.9317 - accuracy: 0.7453
Epoch 173/300
4/4 [==============================] - 0s 11ms/step - loss: 0.8959 - accuracy: 0.7453
Epoch 174/300
4/4 [==============================] - 0s 16ms/step - loss: 0.8945 - accuracy: 0.7736
Epoch 175/300


4/4 [==============================] - 0s 14ms/step - loss: 0.5381 - accuracy: 0.8585
Epoch 248/300
4/4 [==============================] - 0s 11ms/step - loss: 0.5356 - accuracy: 0.8868
Epoch 249/300
4/4 [==============================] - 0s 11ms/step - loss: 0.5341 - accuracy: 0.8679
Epoch 250/300
4/4 [==============================] - 0s 13ms/step - loss: 0.5317 - accuracy: 0.8679
Epoch 251/300
4/4 [==============================] - 0s 17ms/step - loss: 0.5284 - accuracy: 0.8774
Epoch 252/300
4/4 [==============================] - 0s 12ms/step - loss: 0.5288 - accuracy: 0.8868
Epoch 253/300
4/4 [==============================] - 0s 11ms/step - loss: 0.5234 - accuracy: 0.8679
Epoch 254/300
4/4 [==============================] - 0s 10ms/step - loss: 0.5208 - accuracy: 0.8585
Epoch 255/300
4/4 [==============================] - 0s 12ms/step - loss: 0.5221 - accuracy: 0.8491
Epoch 256/300
4/4 [==============================] - 0s 14ms/step - loss: 0.5236 - accuracy: 0.8585
Epoch 257/300


In [21]:
import pyttsx3

In [22]:
#Testing
engine = pyttsx3.init()

while True:
    textList = []
    user_input = input("You: ")
    prediction_input = []

    for letter in user_input:
        if letter not in string.punctuation:
            prediction_input.append(letter.lower())

    prediction_input = ''.join(prediction_input)
    textList.append(prediction_input)
    
    prediction_input = tokenizer.texts_to_sequences(textList)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input], input_shape)
    
    output = model.predict(prediction_input)
    output = output.argmax()
    
    response_tag = le.inverse_transform([output])[0]
    ans = random.choice(responses[response_tag])
    print("Jarvis: ", ans)
    engine.say(ans)
    engine.runAndWait()
    if response_tag == 'goodbye':
        break

You: hi
1/1 [==============================] - 1s 767ms/step
Jarvis:  Hi there, My name is Jarvis! How can I help?
You: track my order
1/1 [==============================] - 0s 34ms/step
Jarvis:  Visit the order page, click on the specific order, select 'track my order', and check the status
You: thnx
1/1 [==============================] - 0s 41ms/step
Jarvis:  Any time!
You: bye
1/1 [==============================] - 0s 29ms/step
Jarvis:  See you later, thanks for visiting. Hope I was able to help!
